In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
from IPython.display import Image

# Statistik

- Pius: Sieg / Niederlage gegen andere KIs & Stockfish auf verschiedenen Leveln
- Pius: Cache Hits / Misses / Tries d.h. Effektivität vom Cache (auch Vergleich mit / ohne Memoisierung)
- Philipp: Zugzeit von verschiedenen KIs und Einstellungen vergleichen
- Philipp: Entwicklung der KIs darstellen
- Anton: Belastung von Arbeitsspeicher in Zusammenhang mit der Tiefe und der Memoisierung (Cache Größe)
- Anton: SVE: Durchschnittliche Tiefe (bezogen auf die maximal Tiefe der Config)
- TBD: Vergleich zwischen Schwarz und Weiß als Farbe

## Ressourcenverbrauch der verschiedenen KI-Versionen

Die CPU-Auslastung ist bei jeder KI-Version maximal, d.h. es wird durchgängig die maximale verfügbare Rechenleistung genutzt. Die KI ist aktuell nicht parallelisiert, d.h. es wird jeweils nur ein CPU-Kern genutzt. Der Festplattenspeicher bewegt sich innerhalb von wenigen MB Verbrauch und ist somit vernachlässigbar. Die kritische Ressource ist der Arbeitsspeicher. Der genutzte Speicher unterscheidet sich je nach KI-Version und Konfiguration stark und ist (neben der Berechnungszeit) ein limitierender Faktor.

### Vergleich des RAM-Verbrauchs

Im Folgenden wird der Verbrauch des Arbeitsspeichers für die KI-Version 5 bis 8 verglichen. Die Versionen 1 bis 4 verfügen noch nicht über das Alpha-Beta-Pruning welches die Berechnungen stark beschleuningt, daher sind sie für einen Vergleich weniger interessant.

#### Messaufbau

Für jede gewählte Konfiguration und KI-Version wurden drei Testspiele gegen die Stockfish-KI gespielt. Die Spiele wurden ohne GUI auf auf einem Debian Linux gestartet. Der virtuelle Server wurde exklusiv für diesen Test erstellt und verfügt über 32 GB Arbeitsspeicher.  
Während den Spielen wurde nach jedem Zug der KI-Version die Größe des Memoisierungs-Caches mithilfe der Funktion `sys.getsizeof()` gemessen. Hierbei ist zu beachten das es sich beim Cache um ein Dictionary handelt welches in Python als Hashtabelle gespeichert wird. Dadurch steigt die Größe des Caches nicht linear an, sondern verdoppelt sich jeweils wenn eine größere Tabelle erzeugt wird.  
Zusätzlich zum Cache wurde der freie Arbeitsspeicher des Gesamtsystems im Abstand von 30 Sekunden gemessen. Der Verbrauch des Systems im Leerlauf wurde anschließend herausgerechnet. Die Messungenauigkeit wird auf etwa 15 MB geschätzt (maximale Abweichungen im Leerlauf).


**Erläuterungen zur Tabelle:**  
Für Cache und RAM wurde jeweils das Maximum ermittelt. Die Exercise05AI besitzt keine Memoisierung, somit gibt es keine Messung für den Cache. Bei der Exercise08AI wurd die maximale Tiefe für die Singular Value Extension jeweils um 1 höher gewählt als die normale Tiefe (minimaler Wert) um so weiterhin vergleichbar zu sein.

| Tiefe | Exercise05AI RAM | Exercise06AI Cache/RAM | Exercise07AI Cache/RAM | Exercise08AI Cache/RAM |
|-------|------------------|------------------------|------------------------|------------------------|
| 3     | 128 MB           | 0,14 MB / 128 MB       | 0,56 MB / 305 MB       | 2,5 MB / 635 MB        |
| 4     | 138 MB           | 10 MB / 305 MB         | 40 MB / 658 MB         | 80 MB / 1010 MB        |
| 5     | 186 MB           | 80 MB / 1189 MB        | 320 MB / 3240 MB       | x MB / x MB            |

Es zeigt sich, dass die Memoisierung den größten Einfluss auf den Arbeitsspeicher hat. Während der Verbrauch ohne Memoisierung nur leicht ansteigt (Exercise05AI), passiert dies deutlich stärker wenn Memoisierung eingesetzt wird (Exercise06AI). Bei der iterativen Suche (Exercise07AI) erhöht sich der Verbrauch abermals. **Ex08AI Betrachtung einfügen**

Das folgende Bild zeigt exemplarisch den Verlauf der RAM-Nutzung für ein Spiel der Exercise08AI gegen Stockfish. Gemessen wurde jeweils im Abstand von 30 Sekunden. Bei den steilen Anstiegen im Diagramm handelt es sich jeweils um eine Folge von Zügen mit vergleichsweise hoher durchschnittlicher Tiefe, d.h. es wurden viele Stellungen untersucht. Der Zeitpunkt zu dem der Cache geleert wurde korreliert mit den Spitzen im Diagramm. Zu diesem Zeitpunkt geht die RAM-Auslastung also wieder stark zurück.

In [ ]:
Image(url="images/RAM_Ex08AI_vs_Stockfish.png", width=1100, height=1100)

Die Daten zeigen einerseits das die Memoisierung viel Arbeitsspeicher erfordert, aber dieser auch effektiv wieder freigegeben wird sobald der Cache durch einen irreversiblen Zug nutzlos wird. Die zweite Tabelle zeigt aber ebenso, dass der Arbeitsspeicher hier durchaus ein Limit für die Suche darstellen kann (wenn auch nur bei einer sehr langen Rechenzeit).